In [ ]:
using ArbNumerics, DifferentialEquations, Format, Integrals, NaturalUnits

using CairoMakie, LaTeXStrings

In [ ]:
set_theme!(theme_latexfonts())

include("tool_script-directory.jl")
include("tool_script-geomspace.jl")

In [ ]:
EU = GeV
NU = NaturalUnit(EU)

In [ ]:
a_ini = ArbFloat(1)
H_ini = ArbFloat(2e-5) * NU.M_Pl

In [ ]:
save_plot_flag = false

In [ ]:
function evolution_system!(du_list, u_list, params, log_a)
    n_inflaton_comoving = EU(exp(u_list[1]), 3)
    ρ_total_comoving = EU(exp(u_list[2]), 4)
    a = exp(log_a)

    m_inflaton = params.m_inflaton
    λ_inflaton_reheaton_reheaton = params.λ_inflaton_reheaton_reheaton

    ρ_inflaton_comoving = min(m_inflaton * n_inflaton_comoving, ρ_total_comoving / a)
    # ρ_reheaton_comoving = ρ_total_comoving - ρ_inflaton_comoving * a
    n_inflaton_comoving = ρ_inflaton_comoving / m_inflaton

    ρ_total = ρ_total_comoving / a^4
    H = sqrt(ρ_total / (3 * NU.M_Pl^2))

    d_n_inflaton_comoving_OVER_d_a = -(λ_inflaton_reheaton_reheaton^2 / (32 * π * m_inflaton * a * H)) * exp(
        π * λ_inflaton_reheaton_reheaton^2 / (m_inflaton^4 * a^3 * H) * n_inflaton_comoving
    ) * n_inflaton_comoving
    d_ρ_total_comoving_OVER_d_a = ρ_inflaton_comoving

    du_list[1] = d_n_inflaton_comoving_OVER_d_a * a / n_inflaton_comoving
    du_list[2] = d_ρ_total_comoving_OVER_d_a * a / ρ_total_comoving

    return du_list
end

In [ ]:
function solve_evolution_system(;
    a_fin=ArbFloat(1e6),
    m_inflaton=GeV(ArbFloat(1e13)),
    λ_inflaton_reheaton_reheaton=GeV(ArbFloat(1e7)),
    solver=Rodas5P(),
    initial_condition_fixing=false
)
    n_inflaton_ini = 3 * NU.M_Pl^2 * H_ini^2 / m_inflaton
    ρ_total_comoving = m_inflaton * n_inflaton_ini * a_ini^4

    n_inflaton_ini = initial_condition_fixing ? min(
        m_inflaton^4 * H_ini / (π * λ_inflaton_reheaton_reheaton^2) * log(
            32 * π * m_inflaton * H_ini / λ_inflaton_reheaton_reheaton^2
        ) * 5,
        n_inflaton_ini
    ) : n_inflaton_ini
        
    evolution_problem = ODEProblem(evolution_system!,
        [
            (log ∘ EUval)(EU, n_inflaton_ini),
            (log ∘ EUval)(EU, ρ_total_comoving),
        ],
        (log(a_ini), log(a_fin)),
        (
            m_inflaton = m_inflaton,
            λ_inflaton_reheaton_reheaton = λ_inflaton_reheaton_reheaton,
        )
    )
    evolution_solution = solve(evolution_problem, solver)
    evolution_solution.retcode == ReturnCode.Success || @warn "ODE return code: $(evolution_solution.retcode)"

    return evolution_solution
end

In [ ]:
total_line_style = (color=:green, linewidth=2);
inflaton_line_style = (color=:blue, linestyle=:dash, linewidth=2);
inflaton_a3_line_style = (color=:gray, linestyle=:dash, linewidth=2);
inflaton_a4_line_style = (color=:blue, linestyle=:dash, linewidth=2);
reheaton_line_style = (color=:red, linestyle=:dash, linewidth=2);
ρ_tot_a4_label = L"\rho_\mathrm{tot.} \times \left( a / a_I \right)^4";
ρ_inf_a4_label = L"\rho_\phi \times \left( a / a_I \right)^4";
ρ_inf_a3_label = L"\rho_\phi \times \left( a / a_I \right)^3";
ρ_reh_a4_label = L"\rho_\varphi \times \left( a / a_I \right)^4";
ρ_tot_label = L"\rho_\mathrm{tot.}";
ρ_inf_label = L"\rho_\phi";
ρ_reh_label = L"\rho_\varphi";

In [ ]:
a_fin = ArbFloat(1e12)
m_inflaton = GeV(ArbFloat(1e13))
λ_inflaton_reheaton_reheaton = GeV(ArbFloat(1e7))

evolution_solution = solve_evolution_system(
    a_fin=a_fin,
    m_inflaton=m_inflaton,
    λ_inflaton_reheaton_reheaton=λ_inflaton_reheaton_reheaton,
    solver=Rosenbrock23(),
    # initial_condition_fixing=true
)

a_list = geomspace(a_ini, a_fin, 1000)
n_inflaton_comoving_list = ((v -> EU(v, 3)) ∘ exp ∘  first ∘ evolution_solution ∘ log).(a_list)
ρ_total_comoving_list = ((v -> EU(v, 4)) ∘ exp ∘ last ∘ evolution_solution ∘ log).(a_list)
ρ_inflaton_comoving_list = m_inflaton .* n_inflaton_comoving_list
ρ_total_list = ρ_total_comoving_list ./ a_list.^4
ρ_inflaton_list = ρ_inflaton_comoving_list ./ a_list.^3
ρ_reheaton_list = max.(ρ_total_list .- ρ_inflaton_list, EU(0, 4))
ρ_reheaton_comoving_list = ρ_reheaton_list .* a_list.^4

figure = Figure(size=(1200, 600), fontsize=24)
axis = Axis(figure[1, 1],
    title="Comoving Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Comoving Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1.1e11), (1e49, 1e75)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:2:100), [L"10^{%$ii}" for ii ∈ 0:2:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_comoving_list); label=ρ_tot_a4_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list .* (a_list ./ a_ini)); label=ρ_inf_a4_label, inflaton_a4_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_comoving_list); label=ρ_reh_a4_label, reheaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list); label=ρ_inf_a3_label, inflaton_a3_line_style...)
axislegend(axis; position=:cb)

axis = Axis(figure[1, 2],
    title="Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1.1e11), (1e30, 1e65)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:3:100), [L"10^{%$ii}" for ii ∈ 0:3:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_list); label=ρ_tot_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_list); label=ρ_inf_label, inflaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_list); label=ρ_reh_label, reheaton_line_style...)
axislegend(axis;)

save_plot_flag && save(joinpath(overleaf_1_directory, "energy_density_evolution_1e13GeV_1e7GeV.pdf"), figure)
figure

In [ ]:
a_fin = ArbFloat(1e12)
m_inflaton = GeV(ArbFloat(1e13))
λ_inflaton_reheaton_reheaton = GeV(ArbFloat(6e7))

evolution_solution = solve_evolution_system(
    a_fin=a_fin,
    m_inflaton=m_inflaton,
    λ_inflaton_reheaton_reheaton=λ_inflaton_reheaton_reheaton,
    solver=Rosenbrock23(),
    # initial_condition_fixing=true
)

a_list = geomspace(a_ini, a_fin, 1000)
n_inflaton_comoving_list = ((v -> EU(v, 3)) ∘ exp ∘  first ∘ evolution_solution ∘ log).(a_list)
ρ_total_comoving_list = ((v -> EU(v, 4)) ∘ exp ∘ last ∘ evolution_solution ∘ log).(a_list)
ρ_inflaton_comoving_list = m_inflaton .* n_inflaton_comoving_list
ρ_total_list = ρ_total_comoving_list ./ a_list.^4
ρ_inflaton_list = ρ_inflaton_comoving_list ./ a_list.^3
ρ_reheaton_list = max.(ρ_total_list .- ρ_inflaton_list, EU(0, 4))
ρ_reheaton_comoving_list = ρ_reheaton_list .* a_list.^4

ρ_inflaton_comoving_freeze_out = m_inflaton^5 * H_ini / (π * λ_inflaton_reheaton_reheaton^2) * log(
    32 * π * m_inflaton * H_ini / (λ_inflaton_reheaton_reheaton^2)
)

figure = Figure(size=(1200, 600), fontsize=24)
axis = Axis(figure[1, 1],
    title="Comoving Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Comoving Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1.2e10), (1e63, 1e74)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
    yminorgridvisible=true, yminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_comoving_list); label=ρ_tot_a4_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list .* (a_list ./ a_ini)); label=ρ_inf_a4_label, inflaton_a4_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_comoving_list); label=ρ_reh_a4_label, reheaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_freeze_out);
    linewidth=2,
    label=L"$\rho_\phi^I$ anal.", color=:orange
)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list); label=ρ_inf_a3_label, inflaton_a3_line_style...)
axislegend(axis; position=:lt)

axis = Axis(figure[1, 2],
    title="Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1.2e10), (1e32, 1e65)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:4:100), [L"10^{%$ii}" for ii ∈ 0:4:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_list); label=ρ_tot_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_list); label=ρ_inf_label, inflaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_list); label=ρ_reh_label, reheaton_line_style...)
axislegend(axis;)

save_plot_flag && save(joinpath(overleaf_1_directory, "energy_density_evolution_1e13GeV_6e7GeV.pdf"), figure)
figure

In [ ]:
a_fin = ArbFloat(1e9)
m_inflaton = GeV(ArbFloat(1e13))
λ_inflaton_reheaton_reheaton = GeV(ArbFloat(1e13))

evolution_solution = solve_evolution_system(
    a_fin=a_fin,
    m_inflaton=m_inflaton,
    λ_inflaton_reheaton_reheaton=λ_inflaton_reheaton_reheaton,
    solver=Rosenbrock23(),
    initial_condition_fixing=true
)

a_list = geomspace(a_ini, a_fin, 1000)
n_inflaton_comoving_list = ((v -> EU(v, 3)) ∘ exp ∘  first ∘ evolution_solution ∘ log).(a_list)
ρ_total_comoving_list = ((v -> EU(v, 4)) ∘ exp ∘ last ∘ evolution_solution ∘ log).(a_list)
ρ_inflaton_comoving_list = m_inflaton .* n_inflaton_comoving_list
ρ_total_list = ρ_total_comoving_list ./ a_list.^4
ρ_inflaton_list = ρ_inflaton_comoving_list ./ a_list.^3
ρ_reheaton_list = max.(ρ_total_list .- ρ_inflaton_list, EU(0, 4))
ρ_reheaton_comoving_list = ρ_reheaton_list .* a_list.^4

ρ_inflaton_comoving_freeze_out = m_inflaton^5 * H_ini / (π * λ_inflaton_reheaton_reheaton^2) * log(
    32 * π * m_inflaton * H_ini / (λ_inflaton_reheaton_reheaton^2)
)

figure = Figure(size=(1200, 600), fontsize=24)
axis = Axis(figure[1, 1],
    title="Comoving Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Comoving Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1e9), (1e45, 1e65)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:2:100), [L"10^{%$ii}" for ii ∈ 0:2:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
    # yminorgridvisible=true, yminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_comoving_list); label=ρ_tot_a4_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list .* (a_list ./ a_ini)); label=ρ_inf_a4_label, inflaton_a4_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_comoving_list); label=ρ_reh_a4_label, reheaton_line_style...)
# lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_freeze_out);
#     linewidth=2,
#     label=L"$\rho_\phi^I$ anal.", color=:orange
# )
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list); label=ρ_inf_a3_label, inflaton_a3_line_style...)
axislegend(axis; position=:rb)

axis = Axis(figure[1, 2],
    title="Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1e9), nothing),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:2:100), [L"10^{%$ii}" for ii ∈ 0:2:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_list); label=ρ_tot_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_list); label=ρ_inf_label, inflaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_list); label=ρ_reh_label, reheaton_line_style...)
axislegend(axis;)

save_plot_flag && save(joinpath(overleaf_1_directory, "energy_density_evolution_1e13GeV_1e13GeV.pdf"), figure)
figure

In [ ]:
a_fin = ArbFloat(1e11)
m_inflaton = GeV(ArbFloat(1e13))
λ_inflaton_reheaton_reheaton = GeV(ArbFloat(1e9))

evolution_solution = solve_evolution_system(
    a_fin=a_fin,
    m_inflaton=m_inflaton,
    λ_inflaton_reheaton_reheaton=λ_inflaton_reheaton_reheaton,
    solver=Rosenbrock23(),
    initial_condition_fixing=true
)

a_list = geomspace(a_ini, a_fin, 1000)
n_inflaton_comoving_list = ((v -> EU(v, 3)) ∘ exp ∘  first ∘ evolution_solution ∘ log).(a_list)
ρ_total_comoving_list = ((v -> EU(v, 4)) ∘ exp ∘ last ∘ evolution_solution ∘ log).(a_list)
ρ_inflaton_comoving_list = m_inflaton .* n_inflaton_comoving_list
ρ_total_list = ρ_total_comoving_list ./ a_list.^4
ρ_inflaton_list = ρ_inflaton_comoving_list ./ a_list.^3
ρ_reheaton_list = max.(ρ_total_list .- ρ_inflaton_list, EU(0, 4))
ρ_reheaton_comoving_list = ρ_reheaton_list .* a_list.^4

ρ_inflaton_comoving_freeze_out = m_inflaton^5 * H_ini / (π * λ_inflaton_reheaton_reheaton^2) * log(
    32 * π * m_inflaton * H_ini / (λ_inflaton_reheaton_reheaton^2)
)

figure = Figure(size=(1200, 600), fontsize=24)
axis = Axis(figure[1, 1],
    title="Comoving Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Comoving Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1.2e10), (1e60, 1e69)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
    yminorgridvisible=true, yminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_comoving_list); label=ρ_tot_a4_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list .* (a_list ./ a_ini)); label=ρ_inf_a4_label, inflaton_a4_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_comoving_list); label=ρ_reh_a4_label, reheaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_freeze_out);
    linewidth=2,
    label=L"$\rho_\phi^I$ anal.", color=:orange
)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_comoving_list); label=ρ_inf_a3_label, inflaton_a3_line_style...)
axislegend(axis; position=:lt)

axis = Axis(figure[1, 2],
    title="Energy Density Evolution",
    xlabel=L"a / a_I",
    ylabel=L"Energy Density $\left[ \mathrm{GeV}^4 \right]$",
    xscale=log10, yscale=log10,
    limits=((1e0, 1.2e10), (1e32, 1e65)),
    xticks=(exp10.(0:100), [L"10^{%$ii}" for ii ∈ 0:100]),
    yticks=(exp10.(0:4:100), [L"10^{%$ii}" for ii ∈ 0:4:100]),
    xminorgridvisible=true, xminorticks=IntervalsBetween(9),
)

lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_total_list); label=ρ_tot_label, total_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_inflaton_list); label=ρ_inf_label, inflaton_line_style...)
lines!(axis, a_list ./ a_ini, EUval.(EU, ρ_reheaton_list); label=ρ_reh_label, reheaton_line_style...)
axislegend(axis;)

save_plot_flag && save(joinpath(overleaf_1_directory, "energy_density_evolution_1e13GeV_1e9GeV.pdf"), figure)
figure